### Simple RAG Application Using Langchain and OpenAI

In [1]:
!pip install langchain -qU
!pip install langchain-hub -qU



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement langchain-hub (from versions: none)

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for langchain-hub


In [2]:
import os

In [3]:
from langchain_openai import ChatOpenAI

c:\Users\msi\AppData\Local\Programs\Python\Python314\Lib\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1
c:\Users\msi\AppData\Local\Programs\Python\Python314\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from dotenv import load_dotenv
import os
from openai import OpenAI

# Load environment variables
load_dotenv()

# Access your API key
api_key = os.getenv("OPENAI_API_KEY")

# Initialize the OpenAI client
client = OpenAI(api_key=api_key)

# Example request
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Hello, OpenAI!"}]
)

print(response.choices[0].message.content)

Hello! How can I assist you today?


In [31]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)   

### Initialized the embedding Model

In [5]:
from langchain_openai import OpenAIEmbeddings
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=api_key)

### Create an Embedding Document

In [6]:
from langchain_core.documents import Document

#define a list of documents with content and metadata
documents = [
    Document(
        page_content="LangChain is a framework for developing applications powered by language models.",
        metadata={"source": "LangChain Docs", "page": 1}
    ),
    Document(
        page_content="OpenAI provides powerful APIs for natural language processing tasks.",
        metadata={"source": "OpenAI Docs", "page": 2}
    ),
    Document(
        page_content="Embeddings are numerical representations of text that capture semantic meaning.",
        metadata={"source": "ML Concepts", "page": 3}
    ),
    Document(
        page_content="Retrieval-Augmented Generation (RAG) combines retrieval of documents with generation of text.",
        metadata={"source": "RAG Paper", "page": 4}
    )
]


In [7]:
documents[0].page_content,documents[0].metadata

('LangChain is a framework for developing applications powered by language models.',
 {'source': 'LangChain Docs', 'page': 1})

In [8]:
pip install -U langchain langchain-community langchain-core


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
# Create a vector store using the documents and the embedding model
from langchain_community.vectorstores import Chroma

In [17]:
from langchain_community.vectorstores import FAISS

In [15]:
vectorstore=FAISS.from_documents(documents, embedding_model)

In [16]:
vectorstore

### Perform Similarity Search

In [21]:
results=vectorstore.similarity_search("What is RAG?")

for result in results:
    print("-----------------")
    print(result.page_content)
    print(result.metadata)
    

-----------------
Retrieval-Augmented Generation (RAG) combines retrieval of documents with generation of text.
{'source': 'RAG Paper', 'page': 4}
-----------------
LangChain is a framework for developing applications powered by language models.
{'source': 'LangChain Docs', 'page': 1}
-----------------
OpenAI provides powerful APIs for natural language processing tasks.
{'source': 'OpenAI Docs', 'page': 2}
-----------------
Embeddings are numerical representations of text that capture semantic meaning.
{'source': 'ML Concepts', 'page': 3}


### Embed Query and Perform Similarity Search by Vector

In [23]:
# Embed a query using the embedding model
query_embedding=embedding_model.embed_query("What is RAG?")
query_embedding[:10]

[0.0006281227106228471,
 0.02569717727601528,
 0.007161187008023262,
 0.03336399793624878,
 -0.031968604773283005,
 -0.027421820908784866,
 -0.005001463927328587,
 -0.04001171141862869,
 0.006894651334732771,
 -0.032391928136348724]

In [25]:
#Print the length of the query embedding
len(query_embedding)

1536

In [26]:
results=vectorstore.similarity_search_by_vector(query_embedding)
for result in results:
    print("-----------------")
    print(result.page_content)
    print(result.metadata)

-----------------
Retrieval-Augmented Generation (RAG) combines retrieval of documents with generation of text.
{'source': 'RAG Paper', 'page': 4}
-----------------
LangChain is a framework for developing applications powered by language models.
{'source': 'LangChain Docs', 'page': 1}
-----------------
OpenAI provides powerful APIs for natural language processing tasks.
{'source': 'OpenAI Docs', 'page': 2}
-----------------
Embeddings are numerical representations of text that capture semantic meaning.
{'source': 'ML Concepts', 'page': 3}


### Create Retriever

In [28]:
# create a retriever from the vector store

retriever=vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k":1}
)

# Perform batch retrieval using the retriever
batch_results=retriever.batch(["What is RAG?","Explain embeddings."])

for result in batch_results:
    print("======== New Query ========")
    for doc in result:
        print("-----------------")
        print(doc.page_content)
        print(doc.metadata)

======== New Query ========
-----------------
Retrieval-Augmented Generation (RAG) combines retrieval of documents with generation of text.
{'source': 'RAG Paper', 'page': 4}
======== New Query ========
-----------------
Embeddings are numerical representations of text that capture semantic meaning.
{'source': 'ML Concepts', 'page': 3}


### Create a Prompt Template

In [30]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Define a message template for the Chat Bot
message="""
Answer the question based on the context below.
{question}

context:
{context}

"""

# create a chat prompt template from messages
prompt=ChatPromptTemplate.from_messages([
    ("human",message)
])

### Chain Retriever and Prompt Template with LLM

In [32]:
chain={"context":retriever,"question":RunnablePassthrough()} | prompt | llm


In [34]:
response=chain.invoke("what is machine learning?")
print(response.content)

Machine learning is a type of artificial intelligence that allows computers to learn and improve from experience without being explicitly programmed.


In [35]:
response=chain.invoke("How are you?")
print(response.content)

I am an AI assistant, so I do not have feelings or emotions. Thank you for asking. How can I assist you today?
